<a href="https://colab.research.google.com/github/aalonsca73/in_silico_toxicology/blob/main/in_silico_toxicology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fastapi kaleido python-multipart uvicorn pubchempy rdkit mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.7 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=72d7c77d4f060a96c26d0e182e68c43c856eaad159f2599df6389dc3035378a0
  Stored in directory: /root/.cache/pi

In [3]:
# Define the filename for the Excel file
filename = 'llistes.xlsx'

# Define the column name to be used as the index.
# Set this variable to the name of the column containing the molecule names of interest.
index = 'Name'

# Define the threshold percentage of zeros to drop columns with a higher percentage.
percent_zeros = 0.7

# Import necessary libraries
import sys
import pubchempy as pcp
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from mordred import Calculator, descriptors

# Create a descriptor calculator with all available descriptors
calc = Calculator(descriptors)

# Read the Excel file using pandas
df = pd.read_excel(filename,sheet_name=1,header=0)
df.set_index(index,inplace=True)

# Create a list to store all canonical SMILES
SMILES_list = []

# Create a list to store molecules not found in PubChem
molecules_not_found = []

# Create a DataFrame to store found molecules
found_molecules_df = pd.DataFrame(columns=['Name','SMILES'])

# Create a new list called data to store all properties
data = []

# Iterate through each molecule identifier in the index column
for ids in df.index:
    try:
        # Fetch the compound name from PubChem
        compound_name = pcp.get_compounds(ids,'name')
        # Check if compound name is empty (no results)
        if not compound_name:
            # Use boolean indexing to drop rows with empty compound_name
            df.drop(ids,axis=0,inplace=True)
            # Add the molecule to the list of molecules_not_found
            molecules_not_found.append(ids)
        else:
            # Extract the first identifier for the compound
            first_identifier = compound_name[0].cid
            # Fetch the canonical SMILES for the first identifier
            first_smiles = pcp.get_compounds(first_identifier,'cid')[0].canonical_smiles
            # Add the canonical SMILES to a list called SMILES_list
            SMILES_list.append(first_smiles)

            # Create a DataFrame with found molecules and add it to the found_molecules_df
            found_molecule_df = pd.DataFrame(data=[[ids, first_smiles]], columns=['Name', 'SMILES'])
            found_molecules_df = pd.concat([found_molecules_df, found_molecule_df], ignore_index=True)

            # Extract properties for the molecule
            try:
                mol = Chem.MolFromSmiles(first_smiles)
                data.append(mol)
            except:
                print(first_smiles)

    except Exception as e:
        print(f"Error for molecule {ids}: {e}")

# Create a DataFrame with molecules not found in PubChem
missing_molecules_df = pd.DataFrame(data=molecules_not_found)

# Reset the index after dropping rows
df.reset_index(inplace=True)

# Remove duplicate canonical SMILES while preserving order
#canonical_SMILES = list(dict.fromkeys(SMILES_list))
seen = set()
canonical_SMILES = [x for x in SMILES_list if not (x in seen or seen.add(x))]



first_smiles_df = pd.DataFrame(data=SMILES_list)
canonical_SMILES_df = pd.DataFrame(data=canonical_SMILES)
first_smiles_df.to_csv('first_smiles.csv',index=False)
canonical_SMILES_df.to_csv('canonical_smiles.csv',index=False)

# We create a DataFrame with canonical SMILES
smiles_df = pd.DataFrame(data=canonical_SMILES)
# Rename the column
smiles_df.columns = ['SMILES']
# Insert the DataFrame with compound names into the DataFrame with SMILES
smiles_df.insert(0,'Name',df['Name'],True)



# Iterate through every SMILES in smiles_df to extract properties of every molecule
#for molecule in smiles_df['SMILES']:
#    try:
#        mol = Chem.MolFromSmiles(molecule)
#        data.append(mol)
#    except:
#        print(molecule)

# Create a new DataFrame called props_df with all obtained molecular properties
props_df = calc.pandas(data)

# Remove uninformative molecular descriptors
# Delete columns where the percentage of zeros exceeds the specified threshold
props_df = props_df.loc[:,(props_df==0).mean()<percent_zeros]
# Filter and keep only columns with numeric data
props_df = props_df.select_dtypes(include=[np.number])

# Merge the DataFrame containing SMILES with the DataFrame containing all properties
names_props = pd.concat([found_molecules_df,props_df],axis=1)

# Create a new document with all information
names_props.to_csv('molecules_with_properties.csv',index=False)

# Create another document with molecules not found in PubChem
missing_molecules_df.to_csv('molecules_not_found.csv',index=False)

100%|██████████| 71/71 [00:19<00:00,  3.69it/s]
